In [4]:
###### Receptive field size computation ######
import math;

########################################################### Calculator
class ReceptiveFieldCalculator():
    def calculate(self, architecture, input_image_size):
        input_layer = ('input_layer', input_image_size, 1, 1, 0.5)
        self._print_layer_info(input_layer)
        
        for key in architecture:
            current_layer = self._calculate_layer_info(architecture[key], input_layer, key)
            self._print_layer_info(current_layer)
            input_layer = current_layer

    def _print_layer_info(self, layer):
        print(f'------')
        print(f'{layer[0]}: n = {layer[1]}; stride = {layer[2]}; receptive field = {layer[3]}; start = {layer[4]}')     
        print(f'------')
            
    def _calculate_layer_info(self, current_layer, input_layer, layer_name):
        n_in = input_layer[1]
        j_in = input_layer[2]
        r_in = input_layer[3]
        start_in = input_layer[4]
        
        k = current_layer[0]
        s = current_layer[1]
        p = current_layer[2]

        n_out = math.floor((n_in - k + 2*p)/s) + 1
        padding = (n_out-1)*s - n_in + k 
        p_right = math.ceil(padding/2)
        p_left = math.floor(padding/2)

        j_out = j_in * s
        r_out = r_in + (k - 1)*j_in
        start_out = start_in + ((k-1)/2 - p_left)*j_in
        return layer_name, n_out, j_out, r_out, start_out
##########################################################


layers = {'conv1' : [3,1,1] ,
    'conv2' : [3,1,1],
    'maxpool1' : [2,2,0],
    'conv3' : [3,1,1] ,
    'maxpool12' : [2,2,0],
    'conv4' : [3,1,1],
    'maxpool23' : [2,2,0],
    'conv5' : [3,1,1] ,
    'maxpool14' : [2,2,0],
    'conv6' : [3,1,1],
    'maxpool51' : [2,2,0],
    'conv7' : [3,1,1],
    'maxpool63' : [2,2,0],
} # {'layer name' : [kernel, stride, padding]} format

calculator = ReceptiveFieldCalculator()
calculator.calculate(layers, 224)

------
input_layer: n = 224; stride = 1; receptive field = 1; start = 0.5
------
------
conv1: n = 224; stride = 1; receptive field = 3; start = 0.5
------
------
conv2: n = 224; stride = 1; receptive field = 5; start = 0.5
------
------
maxpool1: n = 112; stride = 2; receptive field = 6; start = 1.0
------
------
conv3: n = 112; stride = 2; receptive field = 10; start = 1.0
------
------
maxpool12: n = 56; stride = 4; receptive field = 12; start = 2.0
------
------
conv4: n = 56; stride = 4; receptive field = 20; start = 2.0
------
------
maxpool23: n = 28; stride = 8; receptive field = 24; start = 4.0
------
------
conv5: n = 28; stride = 8; receptive field = 40; start = 4.0
------
------
maxpool14: n = 14; stride = 16; receptive field = 48; start = 8.0
------
------
conv6: n = 14; stride = 16; receptive field = 80; start = 8.0
------
------
maxpool51: n = 7; stride = 32; receptive field = 96; start = 16.0
------
------
conv7: n = 7; stride = 32; receptive field = 160; start = 16.0
-

In [1]:
import torch
import argparse
import logging
import pathlib
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch.nn as nn
import torch.nn.functional
from torch.utils.tensorboard import SummaryWriter

import numpy as np

import os.path
import sys


########### Dataset avg and std compute #################

###############################
##### Defining transforms #####
###############################
params = {"padding": (300, 300)}

# pad smaller images to desired size
pad = torchvision.transforms.Pad(padding=params["padding"], fill=0)
# Crop images to be of same 300x300
centercrop = torchvision.transforms.CenterCrop(300)
# Images are B/W but in 3 channels, we only need one
greyscale = torchvision.transforms.Grayscale(num_output_channels=1)
# Negative
invert = torchvision.transforms.functional.invert
# Normalize data
normalize = torchvision.transforms.Lambda(
    lambda x: (x - mean_train_tensor)/std_train_tensor)

# Compose transforms
composed_transforms = torchvision.transforms.Compose([centercrop, greyscale, invert,
                                                      torchvision.transforms.ToTensor()])

# Create transformer class (currently unused, we transform on data loading)


class DatasetTransformer(torch.utils.data.Dataset):

    def __init__(self, base_dataset, transforms):
        self.base_dataset = base_dataset
        self.transform = transforms

    def __getitem__(self, index):
        img, target = self.base_dataset[index]
        return self.transform(img), target

    def __len__(self):
        return len(self.base_dataset)

# Average Std computing tool


def compute_mean_std(loader):
    print("Computing train data mean")
    # Compute the mean over minibatches
    mean_img = None
    i=0
    for imgs, _ in loader:
        i+=1
        if mean_img is None:       
            print("step1")
            mean_img = torch.zeros_like(imgs[0])
            print("step2")
        print("{}/{}".format(i,len(loader)))
        mean_img += imgs.sum(dim=0)
    mean_img /= len(loader.dataset)


    print("Computing train data std")
    # Compute the std over minibatches
    std_img = torch.zeros_like(mean_img)
    for imgs, _ in loader:
        std_img += ((imgs - mean_img)**2).sum(dim=0)
    std_img /= len(loader.dataset)
    std_img = torch.sqrt(std_img)

    # Set the variance of pixels with no variance to 1
    # Because there is no variance
    # these pixels will anyway have no impact on the final decision
    std_img[std_img == 0] = 1

    return mean_img, std_img




########################
##### Loading Data #####
########################
print(os.path.exists("/opt/ChallengeDeep/train/"))
train_path = "/opt/ChallengeDeep/train/"
valid_ratio = 0.2

# Load learning data
print('Data loading started')
train_path = "/opt/ChallengeDeep/train/"
dataset = datasets.ImageFolder(train_path, composed_transforms)
print('Dataset Loaded')

# Train test split
nb_train = int((1.0 - valid_ratio) * len(dataset))
nb_valid = len(dataset)-nb_train
train_dataset, valid_dataset = torch.utils.data.dataset.random_split(dataset, [
                                                                     nb_train, nb_valid])

##### Generating Loaders #####
num_workers = 4
batch_size = 64

# training loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           shuffle=True)

# validation loader
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           shuffle=True)

# Compute avg and std for normalization
mean_train_tensor, std_train_tensor = compute_mean_std(train_loader)
print("mean : {} std : {}".format(mean_train_tensor,std_train_tensor))

True
Data loading started
Dataset Loaded
Computing train data mean
step1
step2
1/10693
2/10693
3/10693
4/10693
5/10693
6/10693
7/10693
8/10693
9/10693
10/10693
11/10693
12/10693
13/10693
14/10693
15/10693
16/10693
17/10693
18/10693
19/10693
20/10693
21/10693
22/10693
23/10693
24/10693
25/10693
26/10693
27/10693
28/10693
29/10693
30/10693
31/10693
32/10693
33/10693
34/10693
35/10693
36/10693
37/10693
38/10693
39/10693
40/10693
41/10693
42/10693
43/10693
44/10693
45/10693
46/10693
47/10693
48/10693
49/10693
50/10693
51/10693
52/10693
53/10693
54/10693
55/10693
56/10693
57/10693
58/10693
59/10693
60/10693
61/10693
62/10693
63/10693
64/10693
65/10693
66/10693
67/10693
68/10693
69/10693
70/10693
71/10693
72/10693
73/10693
74/10693
75/10693
76/10693
77/10693
78/10693
79/10693
80/10693
81/10693
82/10693
83/10693
84/10693
85/10693
86/10693
87/10693
88/10693
89/10693
90/10693
91/10693
92/10693
93/10693
94/10693
95/10693
96/10693
97/10693
98/10693
99/10693
100/10693
101/10693
102/10693
103/10693